In [1]:
import numpy as np

a = 1
b = 2
c = np.array([3, 4])
d = np.array([['a', 'b'], ['c', 'd']], dtype=object)

In [2]:
np.multiply(a, b)  # 数乘 numpy 建议大家使用 `multiply`

2

In [3]:
np.dot(b, c)

array([6, 8])

In [4]:
np.dot(c, d)

array(['aaacccc', 'bbbdddd'], dtype=object)

In [7]:
e = np.array([[1, 2], [3, 4]])

np.dot(d, e)  # 矩阵乘 numpy 建议大家使用 `matmul`

array([['abbb', 'aabbbb'],
       ['cddd', 'ccdddd']], dtype=object)

In [6]:
np.dot(e, d)

array([['acc', 'bdd'],
       ['aaacccc', 'bbbdddd']], dtype=object)

##  [numpy.tensordot](https://www.machenxiao.com/blog/tensordot)

给定两个张量（维度大于等于 $1$ 的数组）`a` 和 `b`，以及一个包含两个数组的数组， (`a_axes`, `b_axes`)，把 `a` 和 `b` 的元素的乘积沿着 `a_axes` 和 `b_axes` 加和。 如果第三个参数是一个常数 `N`，那么就沿着 `a` 的最后 $N$ 个 轴和 `b` 的前 $N$ 个 轴加和。

其实 `tensordot` 无非是将两个操作数整理成了两个矩阵， 然后调用 `dot` 进行了一般的矩阵点乘，再把结果整理成了正确大小的张量。

```py
# nda 是 a 的维数，axes_a 是 a 中要被沿着加和的轴，notin 则是余下的轴
notin = [k for k in range(nda) if k not in axes_a]
# 把要加和的轴连在余下的轴后面
newaxes_a = notin + axes_a
# as_ 是 a.shape，axes_a 是要被加和的轴，所以 N2 是要被加和的那些轴方向上的大小的乘积
N2 = 1
for axis in axes_a:
    N2 *= as_[axis]
# 既然 N2 是新矩阵的列数，新矩阵的行数自然是 a.shape 中剩余元素的乘积
newshape_a = (int(multiply.reduce([as_[ax] for ax in notin])), N2)
# 剩余的轴方向上的大小保留在 olda 数组中
olda = [as_[axis] for axis in notin]
```

举一个例子，比如 `a` 的形状是 `(5, 4, 2, 3)`，要加和的轴是后两轴， 那么 `N2 = 2 * 3 = 6`，最后得到的新矩阵的大小就是 `(20, 6)`。
对于另一个操作数 `b`，tensordot 的处理是完全一致的， 只不过把 `N2` 放在了行数的位置。因为只有这样才能跟 `a` 做矩阵乘法。 由此也可以看到，`a.shape` 中与 `axes_a` 对应位置的元素的乘积 必须和 `b.shape` 中与 `axes_b` 对应位置元素的乘积是一样的。

In [13]:
import numpy as np

a = np.ones([5, 4, 2, 3])
b = np.ones([3, 2, 6])
np.tensordot(a, b, 1).shape

(5, 4, 2, 2, 6)

In [14]:
np.tensordot(a, b, (2, 1)).shape

(5, 4, 3, 3, 6)

In [15]:
np.tensordot(a, b, (3, 0)).shape

(5, 4, 2, 2, 6)

In [16]:
np.tensordot(a, b, ((2, 3), (1, 0))).shape

(5, 4, 6)

In [17]:
np.tensordot(a, b, ((-2, -1), (1, 0))).shape

(5, 4, 6)

In [18]:
a = np.arange(60.).reshape(3,4,5)
b = np.arange(24.).reshape(4,3,2)
c = np.tensordot(a,b, axes=([1,0],[0,1]))
c.shape

(5, 2)

In [19]:
c

array([[4400., 4730.],
       [4532., 4874.],
       [4664., 5018.],
       [4796., 5162.],
       [4928., 5306.]])

In [20]:
# A slower but equivalent way of computing the same...
d = np.zeros((5,2))
for i in range(5):
    for j in range(2):
        for k in range(3):
            for n in range(4):
                d[i,j] += a[k,n,i] * b[n,k,j]

也就是说，tensordot 要求参数满足上面所说的条件，是为了保证能进行这种循环运算。 即使两个 N2 相等，可以经过重新排布之后得到大小正确的张量， tensordot 也只允许用户进行这种能用循环运算表示的点乘， 避免用户（无意中）进行了没有物理意义的运算

In [21]:
a = np.ones([5, 4, 2, 3])
c = np.ones([2, 3, 6])
np.dot(a, c).shape

(5, 4, 2, 2, 6)

In [22]:
np.tensordot(a, c, (-1, -2)).shape

(5, 4, 2, 2, 6)